In [1]:
from src.DQAgent import DQAgent
from src.QAgent import QAgent

## Training and testing the agents 


In [2]:
epsilon = 0.1
alpha = 0.5
gamma = 0.8
agent = QAgent(alpha, gamma, epsilon)
agent.train(500, 'test1')
agent.test_one()

100%|████████████████████████████████████████████████████████████████████████████████| 500/500 [00:05<00:00, 99.62it/s]

Climbed in 161 steps


0

In [5]:
epsilon = 0.7
alpha = 0.0005
gamma = 0.85
agent = DQAgent(alpha, gamma, epsilon)
score, height = agent.train(200)
print("Best training score: {}, height: {}".format(score, height))
agent.save_model('test1')
agent.test_one()


100%|████████████████████████████████████████████████████████████████████████████████| 250/250 [23:29<00:00,  5.64s/it]


Best training score: -199.71401322570517, height: 0.1336622392888327
INFO:tensorflow:Assets written to: ./models/test2\assets
Task failed


0

## Memorising and restoring the saved states

In [17]:
epsilon = 0.1
alpha = 0.5
gamma = 0.8
agent = QAgent(alpha, gamma, epsilon)
agent.load_Q_table('test1')

0

In [ ]:
epsilon = 1
alpha = 0.0001
gamma = 0.8
agent = DQAgent(alpha, gamma, epsilon)
agent.load_model("test1")